In [45]:
#input player, event date
import requests,bs4
from datetime import datetime

#get input of player name and life event date as MM/DD/YYYY
player_tofind='Alaa Abdelnaby'
date_lifeevent='12/01/1990'

#get player URL
first_letter=player_tofind[player_tofind.find(' ')+1].lower()
url='https://www.basketball-reference.com/players/'+first_letter+'/'

#wrap in url2soup?
res=requests.get(url)
bbsoup=bs4.BeautifulSoup(res.text,'lxml')
body=bbsoup.select('tbody')

names=body[0].find_all('tr')

for player_index,player in enumerate(names):
    if player_tofind==str(player.find_all('a')[0].contents[0]):
        print player_index
        urlnew='https://www.basketball-reference.com/'+player.find_all('a')[0].attrs['href']
        break

#get season URL
date_month=float(date_lifeevent[0:date_lifeevent.find('/')])
if date_month>9:
    urlyear=float(date_lifeevent[-4:])+1
else:
    urlyear=float(date_lifeevent[-4:])
urlnew=urlnew[0:-5]+'/gamelog/'+str(int(urlyear))+'/'
urlnew

0
https://www.basketball-reference.com//players/a/abdelal01.html


'https://www.basketball-reference.com//players/a/abdelal01/gamelog/1991/'

In [46]:
res=requests.get(urlnew)
bbsoup=bs4.BeautifulSoup(res.text,'lxml')
body=bbsoup.select('tbody')

games=body[0].find_all('tr')
gsc=[]
for game_number,game in enumerate(games):
    stats=game.find_all('td')
    if len(stats)==29: #skip where he didn't play the game so there's no stats
        #pull out date of game (index 1) and game score (index 27)
        #need to select based on dates
        gsc.append([str(stats[1].contents[0].contents[0]),float(stats[27].contents[0])])

print gsc
#ot working for all? table different?

[]
